In [ ]:
!pip install deap
!pip install scoop

In [ ]:
import collections.abc
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from deap import creator, base, tools, algorithms
from scoop import futures
import random
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import models, layers, optimizers
import time

In [ ]:
dfData = pd.read_csv('/content/drive/MyDrive/GA/bank.csv', sep=';')

In [ ]:
# Encode the classification labels to numbers
# Get classes and one hot encoded feature vectors

le = LabelEncoder()
le.fit(dfData['y'])
allClasses = le.transform(dfData['y'])
allFeatures = dfData.drop(['y'], axis=1)

# Form training, test, and validation sets
X_trainAndTest, X_validation, y_trainAndTest, y_validation = train_test_split(allFeatures, allClasses, test_size=0.20, random_state=40)
X_train, X_test, y_train, y_test = train_test_split(X_trainAndTest, y_trainAndTest, test_size=0.20, random_state=40)

In [ ]:
def getFitness(individual, X_train, X_test, y_train, y_test, user_batch = 1024):
    cols = [index for index in range(len(individual)) if individual[index] == 0]
    X_train_parsed = X_train.drop(X_train.columns[cols], axis=1)
    X_train_oh_features = pd.get_dummies(X_train_parsed)
    X_test_parsed = X_test.drop(X_test.columns[cols], axis=1)
    X_test_oh_features = pd.get_dummies(X_test_parsed)

    shared_features = set(X_train_oh_features.columns) & set(X_test_oh_features.columns)
    remove_from_train = set(X_train_oh_features.columns) - shared_features
    remove_from_test = set(X_test_oh_features.columns) - shared_features
    X_train_oh_features = X_train_oh_features.drop(list(remove_from_train), axis=1)
    X_test_oh_features = X_test_oh_features.drop(list(remove_from_test), axis=1)

    num_cols = X_train_parsed.select_dtypes(include=np.number).columns
    scaler = StandardScaler()
    X_train_num_features = scaler.fit_transform(X_train_parsed[num_cols])
    X_test_num_features = scaler.transform(X_test_parsed[num_cols])

    X_train_features = np.hstack((X_train_oh_features, X_train_num_features))
    X_test_features = np.hstack((X_test_oh_features, X_test_num_features))

    # Build a deep neural network model
    model = models.Sequential()
    model.add(layers.Dense(20, activation='relu', input_shape=(X_train_features.shape[1],)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model and fit it on the training data
    optimizer = optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train_features, y_train, epochs=1, batch_size=user_batch, verbose=0)

    loss, accuracy = model.evaluate(X_test_features, y_test, verbose=0)

    # Return calculated accuracy as fitness
    return (accuracy,)

In [ ]:
# Create Individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Create Toolbox
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(dfData.columns) - 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Continue filling toolbox...
toolbox.register("evaluate", getFitness, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
def getHof():

    # Initialize variables to use eaSimple
    numPop = 100
    numGen = 10
    pop = toolbox.population(n=numPop)
    hof = tools.HallOfFame(numPop * numGen)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=numGen, stats=stats, halloffame=hof, verbose=True)

    return hof

In [ ]:
def getMetrics(hof):

    # Get list of percentiles in the hall of fame
    percentileList = [i / (len(hof) - 1) for i in range(len(hof))]

    # Gather fitness data from each percentile
    testAccuracyList = []
    validationAccuracyList = []
    individualList = []
    for individual in hof:
        testAccuracy = individual.fitness.values
        validationAccuracy = getFitness(individual, X_trainAndTest, X_validation, y_trainAndTest, y_validation)
        testAccuracyList.append(testAccuracy[0])
        validationAccuracyList.append(validationAccuracy[0])
        individualList.append(individual)
    testAccuracyList.reverse()
    validationAccuracyList.reverse()
    return testAccuracyList, validationAccuracyList, individualList, percentileList

In [ ]:
if __name__ == '__main__':

    individual = [1 for i in range(len(allFeatures.columns))]
    testAccuracy = getFitness(individual, X_train, X_test, y_train, y_test)
    validationAccuracy = getFitness(individual, X_trainAndTest, X_validation, y_trainAndTest, y_validation)
    print('\nTest accuracy with all features: \t' + str(testAccuracy[0]))
    print('Validation accuracy with all features: \t' + str(validationAccuracy[0]) + '\n')

    start = time.time()
    hof = getHof()
    testAccuracyList, validationAccuracyList, individualList, percentileList = getMetrics(hof)

    # Get a list of subsets that performed best on validation data
    maxValAccSubsetIndicies = [index for index in range(len(validationAccuracyList)) if validationAccuracyList[index] == max(validationAccuracyList)]
    maxValIndividuals = [individualList[index] for index in maxValAccSubsetIndicies]
    maxValSubsets = [[list(allFeatures)[index] for index in range(len(individual)) if individual[index] == 1] for individual in maxValIndividuals]


    print('\n---Optimal Feature Subset(s)---\n')
    for index in range(len(maxValAccSubsetIndicies)):
        print('Validation Accuracy: \t\t' + str(validationAccuracyList[maxValAccSubsetIndicies[index]]))
        print('Individual: \t' + str(maxValIndividuals[index]))
        print('Number Features In Subset: \t' + str(len(maxValSubsets[index])))
        print('Feature Subset: ' + str(maxValSubsets[index]))

    end = time.time()
    print("Time taken: ",end - start)